In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_df =pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
train_df


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
to_be_dropped = (train_df.isna().sum()[train_df.isna().sum() > 0]/len(train_df))[train_df.isna().sum()[train_df.isna().sum() > 0]/len(train_df) > 0.5].index 

In [6]:
train_df = train_df.drop(to_be_dropped,axis=1)

In [7]:
test_df = test_df.drop(to_be_dropped,axis=1)

In [8]:
train_df.isna().sum()[train_df.isna().sum() > 0]

LotFrontage     259
MasVnrType        8
MasVnrArea        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [10]:
numeric = train_df._get_numeric_data()

In [11]:
numeric.isna().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

In [12]:
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
filled_numeric = pd.DataFrame(imp_median.fit_transform(train_df[numeric.columns]),columns=numeric.columns)

In [13]:
catAttr = list(set(train_df.columns)-set(numeric.columns))


In [14]:
train_df[catAttr].isna().sum()

BsmtExposure      38
MasVnrType         8
BsmtFinType2      38
KitchenQual        0
MSZoning           0
GarageCond        81
ExterQual          0
Exterior2nd        0
Exterior1st        0
PavedDrive         0
HouseStyle         0
SaleType           0
SaleCondition      0
LandSlope          0
Foundation         0
Neighborhood       0
Functional         0
Condition2         0
BsmtQual          37
LandContour        0
Condition1         0
ExterCond          0
RoofStyle          0
Street             0
GarageFinish      81
HeatingQC          0
Utilities          0
GarageType        81
BldgType           0
BsmtCond          37
Electrical         1
GarageQual        81
BsmtFinType1      37
LotShape           0
RoofMatl           0
LotConfig          0
Heating            0
CentralAir         0
FireplaceQu      690
dtype: int64

In [15]:
for att in catAttr:
    print(train_df[att].value_counts(dropna=False))

No     953
Av     221
Gd     134
Mn     114
NaN     38
Name: BsmtExposure, dtype: int64
None       864
BrkFace    445
Stone      128
BrkCmn      15
NaN          8
Name: MasVnrType, dtype: int64
Unf    1256
Rec      54
LwQ      46
NaN      38
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64
TA    735
Gd    586
Ex    100
Fa     39
Name: KitchenQual, dtype: int64
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
TA     1326
NaN      81
Fa       35
Gd        9
Po        7
Ex        2
Name: GarageCond, dtype: int64
TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64
VinylSd    504
MetalSd    214
HdBoard    207
Wd Sdng    197
Plywood    142
CmentBd     60
Wd Shng     38
Stucco      26
BrkFace     25
AsbShng     20
ImStucc     10
Brk Cmn      7
Stone        5
AsphShn      3
Other        1
CBlock       1
Name: Exterior2nd, dtype: int64
VinylSd    515
HdBoard    222
MetalSd    220
Wd Sdng    206
Ply

In [16]:
# for att in catAttr:
#     train_df[att] = train_df[att].apply(lambda x: train_df[att].mode() if np.isnan(x) else x )

In [17]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [18]:
filled_numeric = filled_numeric.drop('Id',axis=1)

In [19]:
X = filled_numeric.drop('SalePrice',axis=1)
Y = filled_numeric['SalePrice']



In [20]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)


In [21]:
from sklearn.model_selection import train_test_split
X_train , X_val , y_train, y_val = train_test_split(X,Y,test_size=0.2,random_state = 42)

In [22]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)

In [23]:
lin_reg.fit(X_train,y_train)
regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=0)

In [24]:
y_fit_0 = lin_reg.predict(X_val)

In [25]:
num_test_df = test_df[test_df._get_numeric_data().columns]

In [26]:
# filled_test_num = imp_median.transform(num_test_df)
num_test_df.fillna(num_test_df.median(),inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [27]:
ids_test = num_test_df['Id']
num_test_df = num_test_df.drop('Id',axis=1)


In [28]:
y_pred_0 = lin_reg.predict(num_test_df)
ypred_regr = regr.predict(num_test_df)

In [29]:
submission = pd.DataFrame({'Id':pd.Series(ids_test),'SalePrice':pd.Series(ypred_regr)})
submission

,Id,SalePrice
0,1461,134836.456653
1,1462,139382.277868
2,1463,159158.387232
3,1464,163704.208447
4,1465,267786.489538
...,...,...
1454,2915,134836.456653
1455,2916,134836.456653
1456,2917,134836.456653
1457,2918,134836.456653


In [30]:
submission.to_csv('./submission.csv',index=False)

In [31]:
submission

,Id,SalePrice
0,1461,134836.456653
1,1462,139382.277868
2,1463,159158.387232
3,1464,163704.208447
4,1465,267786.489538
...,...,...
1454,2915,134836.456653
1455,2916,134836.456653
1456,2917,134836.456653
1457,2918,134836.456653


In [32]:
# pd.read_csv('./submission.csv')